In [1]:
import numpy as np
import tensorflow as tf

In [2]:
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding= 'utf-8') as file:
    text = file.read()

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [7]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [8]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen= max_sequence_len, padding= 'pre'))

In [9]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [10]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [11]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation = 'softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X,y, epochs = 100, verbose= 1)

Epoch 1/100
3010/3010 [==============================] - 150s 49ms/step - loss: 6.2235 - accuracy: 0.0784
Epoch 2/100
3010/3010 [==============================] - 145s 48ms/step - loss: 5.4990 - accuracy: 0.1253
Epoch 3/100
3010/3010 [==============================] - 142s 47ms/step - loss: 5.1173 - accuracy: 0.1487
Epoch 4/100
3010/3010 [==============================] - 147s 49ms/step - loss: 4.7840 - accuracy: 0.1664
Epoch 5/100
3010/3010 [==============================] - 146s 49ms/step - loss: 4.4779 - accuracy: 0.1836
Epoch 6/100
3010/3010 [==============================] - 315s 105ms/step - loss: 4.1925 - accuracy: 0.2037
Epoch 7/100
3010/3010 [==============================] - 163s 54ms/step - loss: 3.9213 - accuracy: 0.2295
Epoch 8/100
3010/3010 [==============================] - 172s 57ms/step - loss: 3.6646 - accuracy: 0.2584
Epoch 9/100
3010/3010 [==============================] - 177s 59ms/step - loss: 3.4219 - accuracy: 0.2924
Epoch 10/100
3010/3010 [=====================

In [22]:
seed_text = "I will talk if they"
next_words = 6

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen= max_sequence_len - 1, padding= 'pre')
    predicted = np.argmax(model.predict(token_list), axis= -1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text+= " " + output_word

print(seed_text)

1/1 [==============================] - 0s 28ms/step
I will talk if they were peculiar alive and my mind
